### Step 1: Import Dependencies

*Make sure to follow the installation first in the readme*

In [1]:
import fiftyone as fo
from fiftyone import ViewField as F
from client.yolov7_client.yolov7_triton_client import YoloV7_Triton_Inference_Client
from client.detectron2_client.detectron2_triton_client import Detectron2_Triton_Client
from ipywidgets import interact, Dropdown

### Step 2: Start Triton Inference Server and CVAT 

(*will likely be combined into single start up script*)

2a. Triton Inference Server: run the following command, replace source with models' path

    docker run --gpus all --rm --ipc=host --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864 \
    -p8000:8000 -p8001:8001 -p8002:8002 --mount type=bind,source="path/to/triton/models",destination=/models \
    nvcr.io/nvidia/tritonserver:22.06-py3 tritonserver --model-repository=/models --strict-model-config=false \
    --log-verbose 1

        The command is as follows:
        * --gpus all: specifies to use all available GPU on device
        * --ipc=host: docker will share resource with host machine
        * --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864: set up container runtime configs
        * -p8000:8000: expose port 8000 for HTTP 
        * -p8001:8001: expose port 8001 for GRPC 
        * -p8002:8002: expose port 8002 for metrics 
        * -mount type=bind,source="path/to/triton/models",destination=/models: mount models to container
        * nvcr.io/nvidia/tritonserver:22.06-py3: pull from triton server image
        * tritonserver --model-repository=/models --strict-model-config=false \
            --log-verbose 1: starts triton inference server
    

2b. CVAT: install local server as per instructions found [here](https://opencv.github.io/cvat/docs/administration/basics/installation/)

   Once installed, to start a local CVAT server at port 8080 run the following commands

    cd /path/to/cvat/clone/ \
    docker compose up -d


In [2]:
# docker run --gpus all --rm --ipc=host --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864 \
# -p8000:8000 -p8001:8001 -p8002:8002 --mount type=bind,source="path/to/triton/models",destination=/models \
# nvcr.io/nvidia/tritonserver:22.06-py3 tritonserver --model-repository=/models --strict-model-config=false \
# --log-verbose 1

### Step 3: cd to baseline directory

In [8]:
# make sure in baseline_system directory
%pwd

'c:\\Users\\Alex Lin\\Desktop\\baseline_system'

### Step 4: Start Fiftyone Instance Locally

Run the following cell, and navigate to localhost:5151

*If the error "Could not connect session, trying again in 10 seconds" occurs, 
it is likely the session is already started.  In that case nagivate to 
localhost:5151 and see if a session is already started*

In [2]:
# start fiftyone

session = fo.launch_app(auto=False)
session.open_tab()


Could not connect session, trying again in 10 seconds

Session launched. Run `session.show()` to open the App in a cell output.


<IPython.core.display.Javascript object>

### Step 5: Initilize the Model Client (and other things)


5a. initialize the variables associated with the model as well as the class labels 

*Note: currently all demo models are trained on coco_classes.  If a custom model implements \
separate class labels, please make sure to replace the classes with the correct class label \
for use in annotation later*

    - url: Inference server URL, default localhost:8001
    - model_info: Print model status, configuration and statistics
    - verbose: Enable verbose client output
    - client_timeout: Client timeout in seconds, default no timeout
    - ssl: Enable SSL encrypted channel to the server
    - root_certificates: File holding PEM-encoded root certificates, default none
    - private_key: File holding PEM-encoded private key, default is none
    - certificate_chain: File holding PEM-encoded certicate chain default is none
    - client_timeout: Client timeout in seconds, default no timeout
    - width: Inference model input width, default 640
    - height: Inference model input height, default 640

In [3]:
url_triton='localhost:8001'
model_info=False
verbose=False
client_timeout=None
ssl=False
root_certificates=None
private_key=None
certificate_chain=None
width=640
height=640
# change this to the class labels your model of choice was trained on.  
# the default demo detectron2 and yolov7 labels are trained on COCO labels
COCO_CLASSES=["person","bicycle","car","motorcycle","airplane","bus","train","truck","boat","traffic light","fire hydrant", 
              "stop sign","parking meter","bench","bird","cat","dog","horse","sheep","cow","elephant","bear","zebra",
              "giraffe","backpack","umbrella","handbag","tie","suitcase","frisbee","skis","snowboard","sports ball","kite",
              "baseball bat","baseball glove","skateboard","surfboard","tennis racket","bottle","wine glass","cup","fork",
              "knife","spoon","bowl","banana","apple","sandwich","orange","broccoli","carrot","hot dog","pizza","donut",
              "cake","chair","couch","potted plant","bed","dining table","toilet","tv","laptop","mouse","remote","keyboard",
              "cell phone","microwave","oven","toaster","sink","refrigerator","book","clock","vase","scissors","teddy bear",
              "hair drier", "toothbrush"]

5b. select the desired client

*If new clients are loaded onto the triton inference server, simply append them \
to the client list and add the conditionals in init_client() to create a client. \
Make sure to import the client*

In [12]:
# initialize the client
client_list = ["select", "yolov7", "detectron2"]
clientW = Dropdown(options=client_list)
client = []

@interact(client_choice=clientW)
def init_client(client_choice):
    """
    Initializes client with choice from dropdown

    :params:
        - client_choice: chosen from client_list
    """
    if client_choice == "select":
        return
    elif client_choice == "yolov7":
        client.append(YoloV7_Triton_Inference_Client(
            url=url_triton,
            model_info=model_info,
            verbose=verbose,
            client_timeout=client_timeout,
            ssl=ssl,
            root_certificates=root_certificates,
            private_key=private_key,
            certificate_chain=certificate_chain,
            width=width,
            height=height
        ))
    elif client_choice == "detectron2":
        client.append(Detectron2_Triton_Client(
            url=url_triton,
            model_info=model_info,
            verbose=verbose,
            client_timeout=client_timeout,
            ssl=ssl,
            root_certificates=root_certificates,
            private_key=private_key,
            certificate_chain=certificate_chain,
            width=1344,
            height=1344
        ))

interactive(children=(Dropdown(description='client_choice', options=('select', 'yolov7', 'detectron2'), value=…

5c. Set Runtime Configs 

*Make sure these configs are for your desired inference mode to avoid errors*

For images
   - input_: Input directory to load from in image
       NOTE: directory must only contain image files
   - fo_dataset: Dataset name to export predictions to fiftyone, 
       default '', no export
   - output_: Output directory, default no output saved
   - tags: list of tags to organize inference results in fiftyone

For videos
   - input_: Input directory to load from in video
       NOTE: directory must only contain video files
   - fo_dataset: Dataset name to export predictions to fiftyone, 
       default '', no export
   - output_: Output directory, default no output saved
   - fps: Video output fps, default 24.0 FPS
   - tags: list of tags to organize inference results in fiftyone

Dummy requires no input 


*Operations to export to fiftyone and export locally can both be down.  Visualization \
(not listed here) is limited and should only be used for development*

In [10]:
input_ = 'C:\\Users\\Alex Lin\\Desktop\\baseline_system\\data\\raw\\images'
output_ = ''
fo_dataset = 'test_run4'
fps = 24.0
tags = ["validation"]

5d. Run Inference in Desired Mode

**If exported to fiftyone, navigate to your fiftyone client (should be a webpage \
at localhost:5151), refresh, and select the dataset exported to to see inference \
results**

In [11]:
inference_choice = ['select', 'image', 'video', 'dummy']
inferenceW = Dropdown(options=inference_choice)

@interact(mode=inferenceW)
def inference(mode):
    """
    Runs inference through Triton Inference Server 

    :params:
        - mode, media type to run through server, chosen from inference_choice
    """
    if mode == 'select':
        return
    elif mode == 'image':
        client[0].infer_image(input_=input_, output_=output_, fo_dataset=fo_dataset, tags=tags)
    elif mode == 'video':
        client[0].infer_video(input_=input_, output_=output_, fo_dataset=fo_dataset, fps=fps, tags=tags)
    elif mode == 'dummy':
        client[0].infer_dummy()

interactive(children=(Dropdown(description='mode', options=('select', 'image', 'video', 'dummy'), value='selec…

### Step 6: Validation Via CVAT

Once inference has completed, validation may be performed through CVAT.  The dataset 
visualized in fiftyone would then be uploaded to CVAT, ground truth can be annotated,
and the result imported back to fiftyone for analysis.

6a. Load the desired dataset (of all available on fiftyone)

In [26]:
datasets = ["select"] + fo.list_datasets()
datasetsW = Dropdown(options=datasets)
# create a global var
dataset = []

@interact(dataset_choice=datasetsW)
def init_dataset(dataset_choice):
    if dataset_choice == "select":
        return
    # load dataset 
    dataset.append(fo.load_dataset(dataset_choice))

interactive(children=(Dropdown(description='dataset_choice', options=('select', 'cvat-annotation-example', 'te…

***
**WARNING: The follow will delete the selected samples from a dataset in fiftyone, \
only run if you are for sure deleting them as they cannot be recovered**

*if error 'name 'session' is not defined' is thrown, restart kernel and rerun \
everything again*

In [16]:
# delete selected samples
delete_view = dataset[0].select(session.selected)
dataset[0].delete_samples(delete_view)

***

6b. Config for Validation

In [14]:
anno_key = "test_run_images33"
label_field="ground_truth"
label_type='detections'
classes=COCO_CLASSES
launch_editor=False
url_cvat="http://localhost:8080"
username="django"
password="bfc"

6c. Run CVAT Validation

Specific Metrics to Pick Data to Validate, **come up with them**


In [ ]:
# create specific view for low confidence model predictions
low_conf_view = (
    dataset[0] \
    .filter_labels("model_detections", F("confidence") < 0.6)
    .sort_by(F("predictions.detections").length(), reverse=True)
    ) \

# fastdup, cleanlab

sample_id = low_conf_view.first().id
view = dataset[0].select(sample_id)

# send samples to CVAT
view.annotate(
    anno_key="test_image_3",
    label_field="model_detections",
    label_type="detections",
    classes=classes,
    launch_editor=True,
    url="http://localhost:8080",
    username="baseline",
    password="arclight"
)

In [25]:

import fiftyone.zoo as foz

dataset = foz.load_zoo_dataset(
    "quickstart", dataset_name="cvat-annotation-example"
)
dataset.persistent = True

dataset.evaluate_detections(
    "predictions", gt_field="ground_truth", eval_key="eval"
)

# Step 2: Locate a subset of your data requiring annotation

# Create a view that contains only high confidence false positive model
# predictions, with samples containing the most false positives first
most_fp_view = (
    dataset
    .filter_labels("predictions", (F("confidence") > 0.8) & (F("eval") == "fp"))
    .sort_by(F("predictions.detections").length(), reverse=True)
)

# Let's edit the ground truth annotations for the sample with the most
# high confidence false positives
sample_id = most_fp_view.first().id
view = dataset.select(sample_id)

# Step 3: Send samples to CVAT

# A unique identifier for this run
anno_key = "cvat_basic_recipe"

view.annotate(
    anno_key,
    label_field="ground_truth",
    attributes=["iscrowd"],
    launch_editor=True,
    url="http://localhost:8080",
    username="bfc",
    password="arclight"
)
print(dataset.get_annotation_info(anno_key))

 100% |████|  187.5Mb/187.5Mb [616.6ms elapsed, 0s remaining, 304.2Mb/s]      
Extracting dataset...
Parsing dataset metadata
Found 200 samples
Dataset info written to 'C:\Users\Alex Lin\fiftyone\quickstart\info.json'
Loading 'quickstart'
 100% |█████████████████| 200/200 [8.7s elapsed, 0s remaining, 15.9 samples/s]      
Dataset 'cvat-annotation-example' created
Evaluating detections...
 100% |█████████████████| 200/200 [47.0s elapsed, 0s remaining, 2.8 samples/s]      
Found existing field 'ground_truth' with multiple types ['detections', 'instances']. Only the 'detections' will be annotated
Computing metadata...
 100% |█████████████████████| 1/1 [17.0s elapsed, 0s remaining, 0.1 samples/s] 
Uploading samples to CVAT...
Arguments the caused this error were:
{'json': {'name': 'FiftyOne_cvat-annotation-example', 'image_quality': 75, 'labels': [{'name': 'airplane', 'attributes': [{'name': 'iscrowd', 'mutable': True, 'input_type': 'text'}, {'name': 'label_id', 'input_type': 'text', 'muta

Exception: 400 error for request <PreparedRequest [POST]> to url http://localhost:8080/api/tasks with the reason Bad Request. Error content: b'{"labels":[{"attributes":[{"values":["This field is required."]},{"values":["This field is required."]}]},{"attributes":[{"values":["This field is required."]},{"values":["This field is required."]}]},{"attributes":[{"values":["This field is required."]},{"values":["This field is required."]}]},{"attributes":[{"values":["This field is required."]},{"values":["This field is required."]}]},{"attributes":[{"values":["This field is required."]},{"values":["This field is required."]}]},{"attributes":[{"values":["This field is required."]},{"values":["This field is required."]}]},{"attributes":[{"values":["This field is required."]},{"values":["This field is required."]}]},{"attributes":[{"values":["This field is required."]},{"values":["This field is required."]}]},{"attributes":[{"values":["This field is required."]},{"values":["This field is requir

6d. Merge Dataset Back to Fiftyone and Cleanup CVAT

In [46]:
# merge annotations back to Fiftyone dataset
dataset[0].load_annotations(anno_key)
dataset[0].load_annotation_view(anno_key)

# Delete tasks from CVAT
results = dataset[0].load_annotation_results(anno_key)
results.cleanup()

# Delete run record (not the labels) from FiftyOne
dataset[0].delete_annotation_run(anno_key)

Download complete
Loading labels for field 'ground_truth'...
 100% |█████████████████████| 0/0 [3.8ms elapsed, ? remaining, ? samples/s] 
Deleting tasks...
 100% |█████████████████████| 1/1 [1.3s elapsed, 0s remaining, 0.8 samples/s] 
